In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# EDF Raw Data Visualizer / Checker

# Inputs
## edf_1 = path to the first .edf EEG file
## edf_2 = path to a second .edf EEG file
## edf_3 = path to a third .edf EEG file
## n_axes = num of total axes. Default = 3

edf_1 = '../data/physionet.org/files/eegmmidb/1.0.0/S001/S001R03.edf'
edf_2 = '../data/physionet.org/files/eegmmidb/1.0.0/S001/S001R07.edf'
edf_3 = '../data/physionet.org/files/eegmmidb/1.0.0/S001/S001R11.edf'
n_axes = 3


def plot_edf_segment(path, ax, seconds=10, max_channels=8):
    raw = mne.io.read_raw_edf(path, preload=False, verbose='ERROR')
    pick_list = np.arange(min(max_channels, raw.info['nchan'])) # safety net for max_channels > nchan
    stop = int(raw.info['sfreq'] * seconds)
    data = raw.get_data(picks=pick_list, start=0, stop=stop)
    times = np.arange(data.shape[1]) / raw.info['sfreq']


    scale = np.nanmax(np.abs(data))
    offsets = np.arange(data.shape[0]) * 0.000584 * 1.2 # 0.000584 derived from commit e007fb1
    for idx, trace in enumerate(data):
        ax.plot(times, trace + offsets[idx], linewidth=0.6)

    ax.set( 
        yticks=offsets,
        yticklabels=[raw.ch_names[p] for p in picks],
        xlabel='Time (s)',
        ylabel='Region',
        title=str(path)[-6:-4],
    )


fig, axes = plt.subplots(1, n_axes, figsize=(12, 6), sharey=True)

plot_edf_segment(edf_1, ax=axes[0])
plot_edf_segment(edf_2, ax=axes[1])
plot_edf_segment(edf_3, ax=axes[2])

fig.tight_layout()
plt.show()


In [ ]:
# EDF matrix generator for task 1 (reference: https://physionet.org/content/eegmmidb/1.0.0/)
# Scans each EDF file with MNE in triplets (corr. to each trail per task 1) for each subject in range. 
# @ Returns matrix with EEG preloaded in memory to be handled by futher operations

# Task 1 (open and close left or right fist)
# Metadata with physionet data:
    # T0 corresponds to rest
    # T1 corresponds to onset of motion (real or imagined)
    # T2 corresponds to onset of motion (real or imagined)

# MNE interpretation: {np.str_('T0'): 1, np.str_('T1'): 2, np.str_('T2'): 3}


task1_subject_matrix = []
events_general, event_id_general = (0, 0)
event_map = {"left": 2, "right": 3} 

for subjectNum in range(1, 3):
    raw_task1_1 = mne.io.read_raw_edf(f"../data/physionet.org/files/eegmmidb/1.0.0/S00{subjectNum}/S00{subjectNum}R03.edf", preload=True)
    raw_task1_2 = mne.io.read_raw_edf(f"../data/physionet.org/files/eegmmidb/1.0.0/S00{subjectNum}/S00{subjectNum}R07.edf", preload=True)
    raw_task1_3 = mne.io.read_raw_edf(f"../data/physionet.org/files/eegmmidb/1.0.0/S00{subjectNum}/S00{subjectNum}R11.edf", preload=True)

    events_general, event_id_general = mne.events_from_annotations(raw_task1_1)
    print(event_id_general)

    subjectEntry = [raw_task1_1, raw_task1_2, raw_task1_3]

    task1_subject_matrix.append(subjectEntry)

task1_subject_matrix


In [ ]:
# @ Returns a 3D array that represents the epochs of every subject across trials.
# len(local_X) = 3, corresponding to C, T, and M. C, T, and M are all arrays. 
# subject_X is a list of all local_X arrays. It is the parent that holds all epochs for a given subject, across trials.
# all_X is a list of all subjects' epochs, parsed per subject themselves.

all_X = [] # X = trials / each n entry into *this* array is an array of every epoch for one subject
all_Y = [] # Y = labels

for subject_entry in task1_subject_matrix:
    subject_X = []
    subject_Y = []
    for raw in subject_entry:
        events, _ = mne.events_from_annotations(raw)
        epochs = mne.Epochs(
            raw,
            events,
            event_id=event_map,
            tmin=0.0,
            tmax=4.0,
            baseline=None,
            preload=True
        )

        data = epochs.get_data()  # (M, C, T)
        local_X = np.transpose(data, (1, 2, 0))  # (C, T, M)

        labels = epochs.events[:, -1]
        local_Y = np.where(labels == 2, -1, 1).reshape(-1, 1)

        subject_X.append(local_X)
        subject_Y.append(local_Y)

    all_X.append(subject_X)
    all_Y.append(subject_Y)

all_X = np.array(all_X, dtype=object)
all_Y = np.array(all_Y, dtype=object)

all_X[0]
